这份代码在QA打分的环节，prompt做了更新，大家可以参考新版本，新版本文件名为`00_PDF解析与QA抽取_v1.1.ipynb`

In [23]:
!pip install langchain langchain_community pypdf openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
%env API_KEY=
%env BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1

In [24]:
import langchain, langchain_community, pypdf, openai

for module in (langchain, langchain_community, pypdf, openai):
    print(f"{module.__name__:<20}{module.__version__}")

langchain           0.2.8
langchain_community 0.2.7
pypdf               4.3.0
openai              1.35.14


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

from datetime import datetime
import os

In [8]:
dt = '20240713'
version = 'v1'

output_dir = os.path.join('outputs', f'{version}_{dt}')
os.makedirs(output_dir, exist_ok=True)

In [12]:
loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

In [13]:
len(documents)

53

In [14]:
len(documents[0].page_content)

643

In [15]:
import re

pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024年"
# re.sub(pattern, '', splitted_docs_large[3].page_content)

In [16]:
merged_docs = [Document(page_content='\n'.join(re.sub(pattern, '', doc.page_content) for doc in documents))]

In [17]:
from uuid import uuid4
import os
import pickle

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs

In [18]:
splitted_docs = split_docs(documents, os.path.join(output_dir, 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)
splitted_docs_large = split_docs(merged_docs, os.path.join(output_dir, 'split_docs_large.pkl'), chunk_size=1500, chunk_overlap=100)

In [19]:
len(splitted_docs), len(splitted_docs_large)

(52, 28)

In [20]:
uuid2doc = {doc.metadata['uuid']: doc.page_content for doc in splitted_docs}
uuid2large_doc = {doc.metadata['uuid']: doc.page_content for doc in splitted_docs_large}

# 抽取QA

In [21]:
qa_gen_prompt_tmpl = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成8个问题、用户回答这个问题的上下文，和基于上下文对问题的回答。

对问题、上下文、答案的要求：

问题要与这段文本相关，不要询问类似“这个问题的答案在哪一章”这样的问题
上下文：上下文必须与原始文本的内容保持一致，不要进行缩写、扩写、改写、摘要、替换词语等
答案：回答请保持完整且简洁，无须重复问题。答案要能够独立回答问题，而不是引用现有的章节、页码等

返回结果以JSON形式组织，格式为[{"question": "...", "context": ..., "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

qa_gen_prompt_tmpl_large_context = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成2个问题，和基于这段文本对问题的回答，回答请保持完整，无须重复问题。
尽可能创建一些需要综合*大段*文本才能回答的问题，但不要问类似“这一段主要讲了什么内容”这样的问题，答案要能够独立回答问题，而不是引用现有的章节、页码等；不要问具体过于细节的问题，例如“海湾国家的2024年预期经济增长率是多少”，而是尽可能问类似“2024年全球经济的几大趋势是什么”、“受局部中东地区紧张局势影响，可能对全球原物料有哪些影响”。
返回结果以JSON形式组织，格式为[{"question": "...", "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

In [25]:
from openai import OpenAI
import time
import random

client = OpenAI(
    api_key=os.environ['API_KEY'],
    base_url=os.environ['BASE_URL']
)

def build_qa_prompt(prompt_tmpl, text):
    prompt = prompt_tmpl.replace('{{document}}', text).strip()
    return prompt

def chat(prompt, max_retry=3, debug=False, top_p=0.95, temperature=0.85):
    def do_chat(prompt):
        completion = client.chat.completions.create(
            model='qwen-long',
            messages=[
                # {"role": "system", "content": "你是一个有用的人工智能助手"},
                {"role": "user", "content": prompt}
            ],
            top_p=top_p,
            temperature=temperature
        )
        return completion.choices[0].message.content

    while max_retry > 0:
        try:
            return do_chat(prompt)
        except Exception as e:
            max_retry -= 1
            sleep_seconds = random.randint(1, 4)
            if debug:
                print(f"{str(e)}, remain retry: {max_retry}, sleeping {sleep_seconds}s {prompt}")
            time.sleep(sleep_seconds)
    return None

In [26]:
text = splitted_docs[40].page_content

In [27]:
print(text)

全球经济金融展望报告
中国银行研究院 40 2024年
注重绿色经济发展，为中海经贸合作创造了良好机遇。
（一）近年来海湾六国经济表现受能源价格影响较大
受国际能源价格走势影响，近年来海湾六国经济增长波动较大。2022年，
在全球油气价格飙升推动下，海湾六国经济增长强劲，实际GDP增速达7.3%。
其中，沙特占据海湾六国经济体量的半壁江山。2022年，沙特GDP规模达1.1
万亿美元，阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元，位列第二
和第三位（图26）。油气部门在海湾六国经济总量中的占比超过40%，在沙特
的占比更是接近七成。
图26：2022年海湾六国经济体量（亿美元）
资料来源：世界银行，中国银行研究院
2023年以来，海湾六国经济增长明显放缓。主要经济体货币政策加速收紧、
全球经济活动减少使全球能源需求下降，海湾六国经济增长面临严重挑战。IMF
在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。其中，
沙特经济增速预计仅为0.8%，低于中东石油出口国的平均水平。相比之下，阿
联酋韧性较强，预计2023年和2024年经济增速分别为3.4%和4.0%，为海湾
六国中最高（图27）。鉴于海湾六国经济结构相对单一，非石油部门虽然为海


In [28]:
print(chat(build_qa_prompt(qa_gen_prompt_tmpl, text), debug=True))

```json
[
    {
        "question": "海湾六国经济增长受什么影响较大？",
        "context": "受国际能源价格走势影响，近年来海湾六国经济增长波动较大。",
        "answer": "国际能源价格走势"
    },
    {
        "question": "2022年海湾六国的平均GDP增速是多少？",
        "context": "2022年，海湾六国经济增长强劲，实际GDP增速达7.3%。",
        "answer": "7.3%"
    },
    {
        "question": "哪个国家在海湾六国中GDP规模最大？",
        "context": "沙特占据海湾六国经济体量的半壁江山。2022年，沙特GDP规模达1.1万亿美元",
        "answer": "沙特"
    },
    {
        "question": "2022年阿联酋的GDP规模大约是多少亿美元？",
        "context": "阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元，位列第二和第三位",
        "answer": "5075亿"
    },
    {
        "question": "2022年沙特油气部门在其经济总量中的占比大约是多少？",
        "context": "油气部门在海湾六国经济总量中的占比超过40%，在沙特的占比更是接近七成。",
        "answer": "接近七成"
    },
    {
        "question": "IMF预测2023年海湾六国的平均经济增速是多少？",
        "context": "IMF在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。",
        "answer": "1.7%"
    },
    {
        "question": "2023年和2024年阿联酋预计的经济增速是多少？",
        "context": "预计2023年和2024年经济增速分别为3.4%和4.0%，为海湾六国中最高。",
        "answer": "2

In [29]:
print(splitted_docs_large[8].page_content)

存在专向性1或属于禁止性补贴，且对欧盟内部产业构成实质性损害的情况下，欧盟即可认定补贴属于
可诉性补贴。在此情况下，欧盟一般可能采取两类反补贴措施：一是加征反补贴税，针对不同类型的
企业，适用的加征税率也有所差异，其中被认定为不合作企业将会被征收最高税率；二是出口商接受
价格承诺，以不低于特定价格出口。反补贴措施一般实施五年。当欧盟认定补贴未对其产业带来损害、
补贴微量或申请人撤诉的情况下，可以无措施结案。
欧盟从2010年首次发起对华反补贴调查开始，共计对华发起过18起反补贴调查，涉及多个制造
业领域，其中接近三分之二的调查最终采取了加征关税的措施。因此，从历史经验看，反补贴调查一
经发起，欧盟较大概率最终将采取加征关税措施。
3.日本经济总体保持稳定，货币政策有望迎来拐点
2023年日本经济整体呈现先升后降走势。上半年，在疫后重新开放市场、
供应链修复等因素影响下，日本经济实现较快增长，一、二季度GDP环比增长
折年率分别达到3.7%和4.5%。但三季度日本经济增长形势急转直下，GDP环
比折年率降至-2.1%，三个季度以来首次出现负增长。从全年情况看，通胀压力
和疲软的日元抑制了私人需求，占日本GDP近六成的个人消费连续两个季度出
现负增长，严重拖累日本经济。前期对日本经济增长贡献较大的外需在三季度
也逐渐失去活力，特别是前期增长旺盛的访日外国游客消费在三季度出现5%
的环比下滑，同期日本进口出现三个季度的首次正增长，净出口对整体经济的
贡献由正转负。民间设备投资在二季度下降1%后三季度再下降0.6%，企业在
价格压力不减的情况下继续削减投资。政府消费经历先降后升走势，日本政府
财政政策在三季度开始发力，但仍不足以支撑日本经济整体表现。按照平均汇
率折算，前三季度日本经济总量被德国赶超，降至全球第四位。
1如果补贴授予当局补贴发放明确限定于特定企业，这种补贴即具有专向性。

图11：日本GDP环比增长折年率变化情况（%）
资料来源：Wind，中国银行研究院
日本通胀压力较大。2022年以来，日本通胀持续上涨，并于2023年初达
到41年来高位。此后受益于能源进口价格下滑以及政府对抑制电费和燃气费的
补贴政策，通胀水平有所回落，但最新数据仍在3%左右的水平震荡，显著高于
过去二十多年0.15%的平均水平。日本对海外能源、食品的依赖程度较为突出，
特别是能源进口依

In [30]:
print(chat(build_qa_prompt(qa_gen_prompt_tmpl_large_context, splitted_docs_large[8].page_content)))

```json
[
  {
    "question": "欧盟在认定补贴属于可诉性补贴后，可能会采取哪些措施？",
    "answer": "欧盟在认定补贴属于可诉性补贴后，一般可能采取加征反补贴税和要求出口商接受价格承诺这两种措施。加征的反补贴税税率根据企业类型不同而有所差异，不合作企业会面临最高税率。价格承诺则意味着出口商必须同意以不低于特定价格出口商品。这些措施通常会实施五年。"
  },
  {
    "question": "2023年日本经济的走势如何？",
    "answer": "2023年日本经济呈现先升后降的走势。上半年，由于疫后市场重新开放、供应链修复等因素，GDP环比增长强劲，但到了三季度，GDP环比折年率转为负增长，出现负2.1%的增长。通胀压力、疲软的日元抑制了私人需求，个人消费连续两个季度下滑，外需也失去了活力，尤其是访日外国游客消费减少，净出口对经济的贡献由正转负。尽管政府财政政策在三季度开始发力，但经济总量仍被德国赶超，降至全球第四位。"
  }
]
```


In [31]:
import threading
import concurrent.futures
from tqdm.auto import tqdm
import json

def gen_qa(splitted_docs, prompt_tmpl, qa_ckpt_filename):
    qa_ckpt = {}
    if os.path.exists(qa_ckpt_filename):
        qa_ckpt = open(qa_ckpt_filename).readlines()
        qa_ckpt = [json.loads(line.strip()) for line in qa_ckpt if line.strip() != '']
        qa_ckpt = {item['uuid']: item for item in qa_ckpt}
        print(f'found checkpoint, item count: {len(qa_ckpt)}')

    file_lock = threading.Lock()
    max_workers = 4
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {doc.metadata['uuid']: executor.submit(chat, build_qa_prompt(prompt_tmpl, doc.page_content), 3, True) for doc in splitted_docs if len(doc.page_content.replace('\n', '')) >= 150 and doc.metadata['uuid'] not in qa_ckpt}
        for uuid in tqdm(futures):
            future = futures[uuid]
            result = future.result()
            if result is None:
                continue

            item = {'uuid': uuid, 'raw_resp': result}
            qa_ckpt[uuid] = item

            # global file_lock
            file_lock.acquire()

            try:
                with open(qa_ckpt_filename, 'a') as f:
                    f.write(json.dumps(item, ensure_ascii=False) + '\n')
            except Exception as e:
                print(e)
            finally:
                file_lock.release()
    return qa_ckpt

In [32]:
detailed_qa_dict = gen_qa(splitted_docs, qa_gen_prompt_tmpl, os.path.join(output_dir, f"qa_ckpt_detailed.jsonl"))

  0%|          | 0/49 [00:00<?, ?it/s]

In [33]:
large_context_qa_dict = gen_qa(splitted_docs_large, qa_gen_prompt_tmpl_large_context, os.path.join(output_dir, f"qa_ckpt_large_context.jsonl"))

  0%|          | 0/28 [00:00<?, ?it/s]

# 后置处理

In [ ]:
import re

def convert2json(text):
    pattern = r'\[.*\]'

    text = text.replace('>>>', '')
    try:
        return json.loads(text)
    except:
        match = re.search(pattern, text, re.DOTALL)
        try:
            matched = match.group(0)
            return json.loads(matched)
        except Exception as e:
            print(f"{match}, {str(e)}")

    return []

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import pickle

def build_qa_df(qa_ckpt, uuid2doc_map):
    data = []

    for key, value in tqdm(qa_ckpt.items()):
        text = value['raw_resp']
        qa_list = convert2json(text)

        for item in qa_list:
            question = item.get('question', '').strip()
            answer = item.get('answer', '').strip()
            context = item.get('context', '').strip()

            if question == '' or answer == '':
                print(qa_list)
                continue
            data.append({
                'uuid': key,
                'question': question,
                'answer': answer,
                'context': context,
                'doc': uuid2doc_map[key]
            })
    qa_df = pd.DataFrame(data)
    return qa_df

In [ ]:
qa_df = build_qa_df(detailed_qa_dict, uuid2doc)
large_context_qa_df = build_qa_df(large_context_qa_dict, uuid2large_doc)

  0%|          | 0/49 [00:00<?, ?it/s]

<re.Match object; span=(8, 1308), match='[\n    {\n        "question": "什么是可诉性补贴？",\n     >, Invalid control character at: line 5 column 63 (char 177)
<re.Match object; span=(8, 1630), match='[\n    {\n        "question": "2024年主要经济体的货币市场利率将>, Expecting value: line 10 column 19 (char 433)
<re.Match object; span=(8, 1288), match='[\n    {\n        "question": "2023年全球大宗商品价格的走势如何>, Invalid control character at: line 5 column 43 (char 144)
<re.Match object; span=(8, 1218), match='[\n  {\n    "question": "2022年RJ/CRB价格指数变动方差是多少？">, Expecting value: line 18 column 17 (char 522)


  0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
len(qa_df)

360

In [ ]:
qa_df.drop_duplicates('question', inplace=True)

In [ ]:
len(qa_df)

350

In [ ]:
len(large_context_qa_df)

56

In [ ]:
large_context_qa_df.drop_duplicates('question', inplace=True)

In [ ]:
len(large_context_qa_df)

56

In [ ]:
qa_df.sample(5)

,uuid,question,answer,context,doc
182,9d41e5b3-2a8c-4584-b7b2-a6b7489758c9,土耳其央行在10月加息了多少个基点？,500个基点,土耳其央行10月大幅加息500个基点,全球经济金融展望报告\n中国银行研究院 23 2024年\n图14：影响国际金融动态的六大因...
161,8cf24c61-8169-41e5-8e05-870bf85a4bc8,中国和巴西在2023年有什么货币合作的进展？,中国与巴西签署了在巴西建立人民币清算安排的合作备忘录，并实现了首次人民币贸易结算全流程闭环操作。,3月，中国与巴西签署在巴西建立人民币清算安排的合作备忘录，10月，两国实现首次人民币贸易结算...,全球经济金融展望报告\n中国银行研究院 21 2024年\n专栏2：全球发展中国家合作进展\...
74,9870421a-c6c9-4ca2-a394-9053497234e5,服务消费对美国前三季私人消费增长的贡献率是多少？,58.7%,服务支出成为消费者支出领域中最稳定和增幅最大的板块，前三季\n度服务消费贡献了私人消费增长的...,全球经济金融展望报告\n中国银行研究院 9 2024年\n综合上述因素，“复苏+分化”将成为...
306,c95e38d5-6741-438f-85e7-7d8f39804b7a,阿美公司的这笔交易金额是多少？,124亿美元,交易金额达124亿美元，,全球经济金融展望报告\n中国银行研究院 43 2024年\n以促进公私合作伙伴关系和放宽外国...
150,3c0da094-8444-428b-9729-f1caa33e4b06,2024年预计新兴经济体的整体经济增速是多少？,4%左右,展望2024年，新兴经济体增长机遇与挑战并存，“南南合作”将成为重要抓手，预计整体经济增速将...,全球经济金融展望报告\n中国银行研究院 19 2024年\n新兴经济体增长的主要因素，三季度...


In [ ]:
qa_df['qa_type'] = 'detailed'

In [ ]:
large_context_qa_df.sample(5)

,uuid,question,answer,context,doc
11,0460392f-a8d7-49e1-8f20-4ef39c163f17,2024年欧洲经济，尤其是欧元区和英国的展望如何？,2024年，欧洲经济增长动能不足，预计疲软表现将延续至上半年。欧元区受到欧洲央行收紧货币政策...,,响2024年美国经济走势的重要因素。2023年美国通胀已明显放缓，受基期效\n应减弱、住宅成...
29,10ca1218-3b51-46c3-b8c1-43521a5d22ed,日本可能在何时退出负利率政策，以及预计的利率调整策略是什么？,预计日本可能在2024年初退出负利率政策，将启动首次加息，但加息幅度较小，调整速度会比较缓慢。,,动性上升，公司债券和公司信贷市场风险、银行间和货币市场风险攀升。之后ROFCI月均值在不稳定...
9,171b54d9-f870-40fe-af92-7d308ee002b6,2024年美国经济可能的走势如何？,预计2024年美国经济增速将进一步回落。存在“软着陆”和衰退两种可能性，但目前“软着陆”的可...,,综合上述因素，“复苏+分化”将成为2024年全球经济主线逻辑，预计全\n年经济增速在2.5%...
44,d98dbebc-1683-410e-9c04-97bfad83d7c1,海湾六国在新能源转型方面有哪些具体的举措？,海湾六国中的多个国家设定了净零排放目标，如阿联酋承诺到2050年实现净零排放，沙特、阿曼、科...,,供了空间，进一步坚定了海湾六国经济转型的决心。\n近年来，海湾六国努力推进新能源转型，增强国...
52,43ec2dde-d2cb-4cce-bc24-55fd5cb341f5,2023年二季度美国办公楼宇的空置率是多少？,2023年二季度美国办公楼宇的空置率为18.2%。,,第七，美联储加息以来住房贷款质量稳定。美国单一家庭房贷整体拖欠率\n季度平均值从2.1%下降...


In [ ]:
large_context_qa_df['qa_type'] = 'large_context'

In [ ]:
qa_df = pd.concat([qa_df, large_context_qa_df])

In [ ]:
qa_df.sample(5)

,uuid,question,answer,context,doc,qa_type
43,61c25726-912a-4bd4-981c-01e9cfa81951,面对经济增长放缓和全球经济挑战，海湾国家采取了什么措施？,为了应对经济增长放缓和全球经济挑战，沙特、阿联酋等国政府大力推动绿色经济转型和经济多元化改革...,,注重绿色经济发展，为中海经贸合作创造了良好机遇。\n（一）近年来海湾六国经济表现受能源价格影...,large_context
16,b78e18df-81e8-44d6-a5ed-800fda878b7d,欧盟在认定补贴属于可诉性补贴后，可能会采取哪些措施？,欧盟在认定补贴属于可诉性补贴后，一般可能采取加征反补贴税和要求出口商接受价格承诺这两种措施。...,,存在专向性1或属于禁止性补贴，且对欧盟内部产业构成实质性损害的情况下，欧盟即可认定补贴属于\...,large_context
287,28c7d4f0-8cea-48b3-9cfa-6683446fd425,海湾六国经济结构的主要问题是什么？,经济结构相对单一,鉴于海湾六国经济结构相对单一，非石油部门虽然为海,全球经济金融展望报告\n中国银行研究院 40 2024年\n注重绿色经济发展，为中海经贸合作...,detailed
41,a9756922-8c03-4a1c-a9f9-02f65b61068d,海湾六国为什么在经济发展上有别于其他中东国家?,海湾六国因为拥有丰富的油气资源，这使得它们在经济发展上积累了大量国民财富，同时，它们的对外开...,,专栏4：大宗商品价格波动与全球金融系统稳定\n近年来，全球大宗商品市场波动加剧。从历年大宗商...,large_context
320,873ee876-8549-49b9-b182-6584113b2701,中国与海湾国家的经贸合作可能如何影响人民币国际化？,通过加强双边经贸合作，人民币跨境贸易支付可能会成为人民币在中东地区使用的主要切入点，进而扩展...,当前，中国与海湾六国拥有紧密的经贸联系，并且随着19个阿拉伯国家相继加入“一带一路”倡议，人...,全球经济金融展望报告\n中国银行研究院 45 2024年\n第四，中海深化经贸合作将为人民币...,detailed


In [ ]:
len(qa_df)

406

# QA质量检查

In [ ]:
# qa_check_prompt_tmpl = """
# 你是一个优秀的NLP方面的助教，你的任务是帮助教授检查期末考试的阅读理解题。
# 你需要根据上下文（<context></context>之间的部分），所出的问题（<question></question>之间的部分），以及答案（<answer></answer>）进行打分。
# 打分需要检查如下3点，并分别对这3点进行打分：
# 1. answer_relevance: 答案相关性，答案是否与问题相关，相关为1，不相关为0
# 2. context_relevance: 上下文相关性，问题和上下文是否相关，相关为1，不相关为0
# 3. groundness: 真实性，检查答案是否是基于上下文回答的，是为1，否为0
# 结果请以JSON形式组织，格式为如下：
# {"answer_relevance": ..., "context_relevance": ..., "groundness": ...}

# 上下文：
# <context>
# {{context}}
# </context>

# 问题：
# <question>
# {{question}}
# </question>

# 答案：
# <answer>
# {{answer}}
# </answer>

# 请打分：
# """

qa_check_prompt_tmpl = """
你是一个优秀的NLP方面的助教，你的任务是帮助检查出题组所出的期末考试的题目。
你需要根据所出的问题（<question></question>之间的部分），以及参考答案（<answer></answer>）进行打分，并给出打分理由，分值是一个int类型的值，取值范围为1-5。
好的问题，应该是询问实时、观点等，而不是类似于“这一段描述了什么”，“文本描述了什么”；
好的答案，应该能够直接回答问题，而不是给出在原文中的引用，例如“第3章”等

结果请以JSON形式组织，格式为如下：
{"score": ..., "reason": ...}

问题：
<question>
{{question}}
</question>

答案：
<answer>
{{answer}}
</answer>

请打分：
"""

In [ ]:
def build_qa_scoring_prompt(row):
    question = row['question']
    answer = row['answer']
    prompt = qa_check_prompt_tmpl.replace('{{question}}', question).replace('{{answer}}', answer)

    return prompt

In [ ]:
import threading
import concurrent.futures
from tqdm.auto import tqdm
import json

qa_scoring_ckpt = {}
qa_scoring_ckpt_filename = os.path.join(output_dir, f"qa_scoring_ckpt.jsonl")
if os.path.exists(qa_scoring_ckpt_filename):
    qa_scoring_ckpt = open(qa_scoring_ckpt_filename).readlines()
    qa_scoring_ckpt = [json.loads(line.strip()) for line in qa_scoring_ckpt if line.strip() != '']
    qa_scoring_ckpt = {item['question']: item for item in qa_scoring_ckpt}
    print(f'found checkpoint, item count: {len(qa_scoring_ckpt)}')

file_lock = threading.Lock()
max_workers = 2
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {row['question']: executor.submit(chat, build_qa_scoring_prompt(row), 3, True) for _, row in qa_df.iterrows() if row['question'] not in qa_scoring_ckpt}
    for question in tqdm(futures):
        future = futures[question]
        result = future.result()
        if result is None:
            continue

        item = {'question': question, 'raw_resp': result}
        qa_scoring_ckpt[question] = item

        global file_lock
        file_lock.acquire()

        try:
            with open(qa_scoring_ckpt_filename, 'a') as f:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        except Exception as e:
            print(e)
        finally:
            file_lock.release()

  0%|          | 0/405 [00:00<?, ?it/s]

In [ ]:
qa_scoring_dict = {}
for key, value in qa_scoring_ckpt.items():
    try:
        qa_scoring_dict[key] = json.loads(value['raw_resp'].replace('”}', '"}'))
        if 'score' not in qa_scoring_dict[key]:
            raise ValueError(f'no score in result, question: {key}')
    except Exception as e:
        print(f"{key}, error: {e}")

In [ ]:
qa_df['score'] = qa_df['question'].apply(lambda q: qa_scoring_dict[q]['score'])
qa_df['score_reason'] = qa_df['question'].apply(lambda q: qa_scoring_dict[q]['reason'])

In [ ]:
qa_df.sample(5)

,uuid,question,answer,context,doc,qa_type,score,score_reason
197,a52f60d5-4430-405f-a2dd-8a975ac03b91,ROFCI在哪一区域表明金融市场不稳定？,不稳定区域,仍然处于不稳定区域，但是月度指标及季度平均值在全年呈波动下降趋势。,全球经济金融展望报告\n中国银行研究院 25 2024年\n图15：美联储资产规模变化（亿美...,detailed,4,问题询问的是具体区域，答案直接给出了相应答案，但'不稳定区域'可能需要更多的上下文信息来精确指代。
343,c47057d6-35c0-4466-bafd-4ed036030c1d,在2024年有多少比例的办公空间租赁合同将到期？,17%,预计美国17%的办公空间租赁合同将在2024年底前到期,全球经济金融展望报告\n中国银行研究院 47 2024年\n第七，美联储加息以来住房贷款质量...,detailed,5,问题询问的是具体数据，具有实现实时查询的价值；答案直接给出了一个明确的比例，无需额外查找信息...
5,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,展望2024年，全球经济复苏的预期如何？,全球经济复苏预计将依旧疲软。,展望2024年，预计全球经济复苏将依旧疲软,研究院\n全球经济金融展望报告\n要点2024年年报（总第57期） 报告日期：2023年12...,detailed,5,问题询问的是未来的经济预期，属于观点和预测的范畴，符合优质问题的标准。答案直接给出了对经济复...
196,a52f60d5-4430-405f-a2dd-8a975ac03b91,2023年1-11月，美国金融危机风险指标的平均值是多少？,42.33,2023年1-11月，美国金融危机风险指标（ROFCI）的月度平均值从2022年的42.83...,全球经济金融展望报告\n中国银行研究院 25 2024年\n图15：美联储资产规模变化（亿美...,detailed,5,问题询问的是具体的数据，需要考生对实时信息有了解，答案直接给出了数值，符合要求。
34,56a6c642-9f39-413e-93dd-362b30d9fa5b,2024年全球股票市场有哪些主要特征？,2024年全球股票市场的主要特征包括：美国股票市场受到美国通胀回落、美联储暂停加息及企业盈利...,,图19：美国联邦基金目标利率与全球MSCI指数\n资料来源：Wind，中国银行研究院\n表3...,large_context,5,问题询问的是未来全球股票市场的特征，要求有分析和预测，答案提供了具体的特征，并且给出了支持这...


In [ ]:
qa_df['score'].value_counts()

score
5    326
4     64
3     15
2      1
Name: count, dtype: int64

In [ ]:
qa_df[qa_df['score'] == 3]

,uuid,question,answer,context,doc,qa_type,score,score_reason
25,1b782370-c7f8-48a8-9647-591bb91f27f7,报告中提到的主要经济体GDP增速变化趋势的图的名称是什么？,主要经济体GDP增速变化趋势,图2：主要经济体GDP增速变化趋势（%）,全球经济金融展望报告\n中国银行研究院 3 2024年\n图2：主要经济体GDP增速变化趋势...,detailed,3,问题要求回答图表的名称，而答案直接复述了问题中的内容，没有提供具体的信息。一个好的答案应当是...
33,ded827f2-f0de-429b-9912-344018765257,图3展示了什么内容的变化趋势？,部分经济体工业生产指数,图3：部分经济体工业生产指数变化趋势（2010年=100）,全球经济金融展望报告\n中国银行研究院 4 2024年\n图3：部分经济体工业生产指数变化趋...,detailed,3,问题要求描述图表内容的变化趋势，但没有询问具体的细节或需要分析。答案直接给出了图表的主题，没...
40,4bd96918-ce66-4762-a725-8a7e4b543dbe,图4展示了什么数据？,部分欧洲国家的零售销售指数,图4：部分欧洲国家零售销售指数,全球经济金融展望报告\n中国银行研究院 5 2024年\n图4：部分欧洲国家零售销售指数\n...,detailed,3,问题要求回答图4展示的具体内容，答案直接提供了信息，但没有提及是否是根据图4得出的结论，没有...
50,2e253555-a34a-48cb-824b-da74f33a9111,图6中的数据来源是哪里？,IMF，中国银行研究院,图6：全球投资率变化趋势（%） 资料来源：IMF，中国银行研究院,全球经济金融展望报告\n中国银行研究院 6 2024年\n2023年全球跨境直接投资将继续下...,detailed,3,问题要求回答数据来源，但没有提供足够的上下文使得答案能在不查看图6的情况下被理解。同时，答案...
70,774ab320-aba8-45a7-a5f4-791782b46d08,哪些因素将对经济形成托底，减缓经济下行幅度？,一些稳定因素,与此同时，一些稳定因素将逐渐显现，对经济形成托底，减缓经济下行幅度。,全球经济金融展望报告\n中国银行研究院 8 2024年\n率政策。\n二是各国经济走势将进一...,detailed,3,问题询问的是经济的托底因素，具有开放性和探讨性，是一个不错的问题。但答案'一些稳定因素'过于...
143,733c7f0b-06fd-40f4-a1de-0c863db6069a,哪些因素可能支撑亚太新兴经济体的内需？,信息不足，无法直接回答，需要更多上下文信息。,不过强劲的内需是支撑亚太,全球经济金融展望报告\n中国银行研究院 18 2024年\n图12：日本通胀同比增速变化（%...,detailed,3,问题询问的是支撑亚太新兴经济体内需的因素，这是一个开放性问题，可以探讨多种可能性，但答案表示...
153,883b0471-5082-44f4-b625-86386102bc2d,图13展示了什么内容？,部分亚洲新兴经济体GDP同比增速,图13：部分亚洲新兴经济体GDP同比增速（%）,全球经济金融展望报告\n中国银行研究院 20 2024年\n图13：部分亚洲新兴经济体GDP...,detailed,3,问题要求描述图片内容，但更好的问题是引导学生分析或解释图表的意义，而不仅仅是复述。答案直接回...
183,9d41e5b3-2a8c-4584-b7b2-a6b7489758c9,俄罗斯央行在何时加息了200个基点？,未在文中给出具体时间,俄罗斯央行加息200个基点,全球经济金融展望报告\n中国银行研究院 23 2024年\n图14：影响国际金融动态的六大因...,detailed,3,问题询问的是具体事件的时间，属于事实性问题，但答案指出信息未提供，说明题目可能缺乏适当的上下...
194,a52f60d5-4430-405f-a2dd-8a975ac03b91,图15展示的是什么内容？,美联储资产规模变化,图15：美联储资产规模变化（亿美元）\n资料来源：美联储，中国银行研究院,全球经济金融展望报告\n中国银行研究院 25 2024年\n图15：美联储资产规模变化（亿美...,detailed,3,问题要求描述图片内容，但优秀的问题应更倾向于询问实时信息或观点。答案直接回答了问题，但没有提...
225,65b167b2-bf81-4148-b6a6-7afb9946676d,报告讨论的时间范围是什么？,2024年,全球经济金融展望报告 中国银行研究院 30 2024年,全球经济金融展望报告\n中国银行研究院 30 2024年\n图18：主要货币兑美元汇率较20...,detailed,3,问题询问的是时间范围，但答案只给出了一个具体的年份，没有明确是否是开始或结束时间，也没有提供...


In [ ]:
qa_df[qa_df['score'] == 2]

,uuid,question,answer,context,doc,qa_type,score,score_reason
91,2800873d-c399-4c23-9721-9238e2c941f3,消费者借贷能力和意愿受到什么因素的影响？,美国家庭债务余额拖欠率回升至3%,美国家庭债务余额拖欠率回升至3%，消费者借贷能力和意愿将有所下降。,全球经济金融展望报告\n中国银行研究院 11 2024年\n响2024年美国经济走势的重要因...,detailed,2,问题询问的是影响消费者借贷能力和意愿的因素，而答案提供的是一条关于美国家庭债务余额拖欠率的具...


In [ ]:
qa_df[qa_df['score'] == 1]

,uuid,question,answer,context,doc,qa_type,score,score_reason


In [ ]:
hq_qa_df = qa_df[qa_df['score'] >= 4]

In [ ]:
hq_qa_df.shape

(390, 8)

In [ ]:
test_q = hq_qa_df.sample(100, replace=False)['question'].values.tolist()

In [ ]:
hq_qa_df['dataset'] = 'train'
hq_qa_df.loc[hq_qa_df['question'].isin(test_q), 'dataset'] = 'test'

/tmp/ipykernel_3123169/2938610661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hq_qa_df['dataset'] = 'train'


In [ ]:
hq_qa_df.sample(10)

,uuid,question,answer,context,doc,qa_type,score,score_reason,dataset
154,883b0471-5082-44f4-b625-86386102bc2d,在当前背景下，哪些国家需要发展可再生能源？,沙特阿拉伯、阿联酋、科威特,沙特阿拉伯、阿联酋、科威特等国依赖油气出口的经济增长模式难以为继，上述国家逐渐加快推动可再生...,全球经济金融展望报告\n中国银行研究院 20 2024年\n图13：部分亚洲新兴经济体GDP...,detailed,4,问题询问的是在当前背景下需要发展可再生能源的国家，答案直接给出了三个具体的国家名称，直接回答...,test
303,c6fc6079-0daf-4d6a-bd6e-7d9820012425,阿联酋计划到哪一年将吸引外资规模提升至1500亿美元？,2031年,以期到2031年将吸引外资规模提升至1500亿美元。,全球经济金融展望报告\n中国银行研究院 42 2024年\n供了空间，进一步坚定了海湾六国经...,detailed,5,问题询问的是具体的时间信息，是一个有明确答案的问题；答案直接给出了具体年份，符合要求，没有提...,train
300,c6fc6079-0daf-4d6a-bd6e-7d9820012425,阿曼的绿氢战略预计需要多少投资额？,1400亿美元,阿曼推出的绿氢战略计划到2050年投资1400亿美元。,全球经济金融展望报告\n中国银行研究院 42 2024年\n供了空间，进一步坚定了海湾六国经...,detailed,5,问题询问具体的数据，是开放性问题；答案直接给出了数值，无需查找原文，符合好答案的要求。,train
90,2800873d-c399-4c23-9721-9238e2c941f3,高利率对美国家庭债务负担有什么影响？,增加了美国家庭债务负担,2023年利率上升增加了美国家庭债务负担，家庭债务支出/可支配收入虽低于历史水平，但呈现上升态势。,全球经济金融展望报告\n中国银行研究院 11 2024年\n响2024年美国经济走势的重要因...,detailed,5,问题询问的是利率变化对美国家庭债务的实际影响，这是一个开放式的问题，需要理解并分析经济原理。...,train
167,8cf24c61-8169-41e5-8e05-870bf85a4bc8,中国的“一带一路”合作文件覆盖了多少国家和人口的比例？,覆盖了全球超过七成的国家和六成的人口。,152个国家和32个国际组织已与中国签署“一带一路”合作文件，覆盖全球超过七成的国家和六成的人口。,全球经济金融展望报告\n中国银行研究院 21 2024年\n专栏2：全球发展中国家合作进展\...,detailed,5,问题询问的是具体的数据比例，答案直接给出了准确的比例，没有引用原文，符合好的问题和答案的标准。,train
164,8cf24c61-8169-41e5-8e05-870bf85a4bc8,中国在绿色发展方面的举措是什么？,中国将“促进绿色发展”列为高质量共建“一带一路”行动之一，并计划在非洲和东南亚成立区域办公室...,中国将“促进绿色发展”列为高质量共建“一带一路”八项行动之一，将在非洲和东南亚成立区域办公室...,全球经济金融展望报告\n中国银行研究院 21 2024年\n专栏2：全球发展中国家合作进展\...,detailed,4,问题询问的是具体的举措，答案直接提供了中国的绿色发展措施，内容具体且直接回答了问题。但答案稍...,train
159,883b0471-5082-44f4-b625-86386102bc2d,地缘政治冲突对全球经济运行产生了什么影响？,全球经济运行的不确定性增加,当前，俄乌、巴以等地缘政治冲突持续，全球经济运行的不确定性增加。,全球经济金融展望报告\n中国银行研究院 20 2024年\n图13：部分亚洲新兴经济体GDP...,detailed,4,问题询问的是具体的影响，答案直接给出了因地缘政治冲突带来的经济影响，即增加了全球经济运行的不...,train
6,e73a0c9d-d42b-4350-a4c3-b38bf67c68a5,哪些热点问题在报告中被提及？,海湾六国经济发展与投资前景，高利率和高债务对美国房地产市场脆弱性的影响。,海湾六国经济发展与投资前景、高利率和高债务对美国房地产市场脆弱性的影响等热点问题值得关注。,研究院\n全球经济金融展望报告\n要点2024年年报（总第57期） 报告日期：2023年12...,detailed,4,问题询问的是具体被提及的热点问题，答案直接给出了这些问题的名称，没有直接引用原文，回答具有针...,test
262,d58699ac-79da-4321-98c5-56c0350d2ba5,美欧加大国债发行力度可能导致什么后果？,增加财政体系的脆弱性,未来随着美欧加大国债发行力度，将进一步增加财政体系的脆弱性。,全球经济金融展望报告\n中国银行研究院 35 2024年\n体国债收益率出现较大幅度下跌。当...,detailed,5,问题询问的是国债发行可能带来的后果，这需要考生理解和分析经济政策的影响。答案直接给出了一个合...,test
146,3c0da094-8444-428b-9729-f1caa33e4b06,2023年前两个季度巴西的GDP增速超过了多少？,3%,2023年前两个季度巴西和墨西哥GDP增速均超过3%，阿根廷则面临高通胀与负增长并存的困境。,全球经济金融展望报告\n中国银行研究院 19 2024年\n新兴经济体增长的主要因素，三季度...,detailed,5,问题询问具体的数据，是可验证的事实；答案直接给出了数值，符合要求。,train


In [ ]:
hq_qa_df.to_excel(os.path.join(output_dir, f'question_answer.xlsx'), index=False)